# NBA Machine Learning

The objective for our project will be to create a deep learning model that will be able to predict future outcomes of a basketball season. 
This is done because this could be used as a viable tool in the process of sports betting. The model would function almost exactly the same as one that predicts stock price and signals when to buy or sell, the only difference is that it would predict a player's stat and would signal when to bet instead of buying/selling.

Our project will use historical data from NBA teams such as previous season's records, shot making percentages, defensive ratings, 
and several others from the last ten seasons in a deep learning model to predict the next season's standings for each season and individual player statistics.


In [1]:
#import libraries required to run program
import numpy as np
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

2023-06-03 13:42:51.362297: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#import csv containing Jokic's statistics
jokic_csv_path = Path("Resources/jokic_2023.csv")
jokic_df = pd.read_csv(jokic_csv_path)
jokic_df.head()

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1.0,2022-10-19,27-242,DEN,@,UTA,L (-21),1,33:25,...,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0
1,2,2.0,2022-10-21,27-244,DEN,@,GSW,W (+5),1,34:24,...,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0
2,3,3.0,2022-10-22,27-245,DEN,NaN,OKC,W (+5),1,38:41,...,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0
3,4,4.0,2022-10-24,27-247,DEN,@,POR,L (-25),1,26:58,...,8.0,9.0,9.0,0.0,0.0,1.0,5.0,9.0,13.8,-10.0
4,5,5.0,2022-10-26,27-249,DEN,NaN,LAL,W (+11),1,34:48,...,12.0,13.0,9.0,4.0,0.0,3.0,3.0,31.0,34.3,28.0


In [3]:
#remove rows with games Jokic did not play
jokic_df = jokic_df[jokic_df['GS'] != "Inactive"]
display(jokic_df)

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1.0,2022-10-19,27-242,DEN,@,UTA,L (-21),1,33:25,...,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0
1,2,2.0,2022-10-21,27-244,DEN,@,GSW,W (+5),1,34:24,...,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0
2,3,3.0,2022-10-22,27-245,DEN,NaN,OKC,W (+5),1,38:41,...,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0
3,4,4.0,2022-10-24,27-247,DEN,@,POR,L (-25),1,26:58,...,8.0,9.0,9.0,0.0,0.0,1.0,5.0,9.0,13.8,-10.0
4,5,5.0,2022-10-26,27-249,DEN,NaN,LAL,W (+11),1,34:48,...,12.0,13.0,9.0,4.0,0.0,3.0,3.0,31.0,34.3,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,73,65.0,2023-03-22,28-031,DEN,@,WAS,W (+14),1,33:36,...,7.0,12.0,7.0,3.0,0.0,2.0,1.0,31.0,35.7,28.0
73,74,66.0,2023-03-25,28-034,DEN,NaN,MIL,W (+23),1,31:53,...,5.0,6.0,11.0,1.0,0.0,3.0,2.0,31.0,27.7,19.0
74,75,67.0,2023-03-27,28-036,DEN,NaN,PHI,W (+5),1,34:58,...,14.0,17.0,12.0,0.0,2.0,3.0,3.0,25.0,31.6,11.0
78,79,68.0,2023-04-04,28-044,DEN,@,HOU,L (-21),1,24:41,...,8.0,10.0,4.0,2.0,3.0,8.0,1.0,14.0,9.6,-21.0


In [4]:
#dropped columns that were not relevant to model
jokic_df.drop(columns= ["Rk","Tm","Date","Age","Opp"], axis=1, inplace = True)
jokic_df.head()

,G,Unnamed: 5,Unnamed: 7,GS,MP,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1.0,@,L (-21),1,33:25,12.0,17.0,0.706,1.0,3.0,...,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0
1,2.0,@,W (+5),1,34:24,7.0,13.0,0.538,1.0,2.0,...,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0
2,3.0,NaN,W (+5),1,38:41,6.0,10.0,0.600,1.0,1.0,...,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0
3,4.0,@,L (-25),1,26:58,3.0,4.0,0.750,0.0,0.0,...,8.0,9.0,9.0,0.0,0.0,1.0,5.0,9.0,13.8,-10.0
4,5.0,NaN,W (+11),1,34:48,12.0,17.0,0.706,0.0,4.0,...,12.0,13.0,9.0,4.0,0.0,3.0,3.0,31.0,34.3,28.0


In [5]:
#added titles to columns without names
jokic_df.rename(columns={'Unnamed: 5':'Home_or_Away'},inplace= True)
jokic_df.rename(columns={'Unnamed: 7':'MOV'},inplace= True)

In [6]:
#changed Home or Away coumn so that if the game was at home it is a value of 0 and if away it is 1
jokic_df['Home_or_Away'] = jokic_df['Home_or_Away'].replace('@', 1)
jokic_df = jokic_df.fillna(0)

In [7]:
#convert minutes played column to seconds played 
def convert_to_numerical(time_obj):
    minutes, seconds = time_obj.split(':')
    total_seconds = int(minutes) * 60 + int(seconds)
    return total_seconds

jokic_df['MP'] = jokic_df['MP'].apply(convert_to_numerical)
jokic_df.rename(columns={'MP':'Seconds_played'},inplace =True)
jokic_df.head()

,G,Home_or_Away,MOV,GS,Seconds_played,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1.0,1.0,L (-21),1,2005,12.0,17.0,0.706,1.0,3.0,...,2.0,4.0,6.0,3.0,0.0,3.0,4.0,27.0,24.5,-5.0
1,2.0,1.0,W (+5),1,2064,7.0,13.0,0.538,1.0,2.0,...,9.0,12.0,10.0,0.0,0.0,5.0,4.0,26.0,24.9,-3.0
2,3.0,0.0,W (+5),1,2321,6.0,10.0,0.600,1.0,1.0,...,15.0,16.0,13.0,1.0,1.0,3.0,2.0,19.0,25.4,18.0
3,4.0,1.0,L (-25),1,1618,3.0,4.0,0.750,0.0,0.0,...,8.0,9.0,9.0,0.0,0.0,1.0,5.0,9.0,13.8,-10.0
4,5.0,0.0,W (+11),1,2088,12.0,17.0,0.706,0.0,4.0,...,12.0,13.0,9.0,4.0,0.0,3.0,3.0,31.0,34.3,28.0


In [8]:
#converted columns with float64 types to int64 types
jokic_int_df = jokic_df[['G', 'Home_or_Away', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-']].round().astype('int64')


In [9]:
#concat dataframes that contained all columns
jokic_df.drop(columns= ['G', 'Home_or_Away', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'GmSc', '+/-'], axis=1, inplace = True)
jokic_full_df = pd.concat([jokic_df, jokic_int_df], axis=1)
jokic_full_df.tail()

,MOV,GS,Seconds_played,G,Home_or_Away,FG,FGA,FG%,3P,3PA,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
72,W (+14),1,2016,65,1,12,16,1,2,3,...,7,12,7,3,0,2,1,31,36,28
73,W (+23),1,1913,66,0,10,20,0,1,6,...,5,6,11,1,0,3,2,31,28,19
74,W (+5),1,2098,67,0,8,11,1,0,0,...,14,17,12,0,2,3,3,25,32,11
78,L (-21),1,1481,68,1,7,13,1,0,2,...,8,10,4,2,3,8,1,14,10,-21
80,L (-4),1,1598,69,1,2,5,0,0,1,...,10,10,10,2,0,4,2,6,10,-5


In [10]:
jokic_full_df['WinOrLoss'] = jokic_full_df['MOV'].str[0].map({'L':0,'W':1})
jokic_full_df['Margin'] = jokic_full_df['MOV'].str[1:].str.extract("(\d+)").astype(int)
jokic_full_df.dtypes

MOV               object
GS                object
Seconds_played     int64
G                  int64
Home_or_Away       int64
FG                 int64
FGA                int64
FG%                int64
3P                 int64
3PA                int64
3P%                int64
FT                 int64
FTA                int64
FT%                int64
ORB                int64
DRB                int64
TRB                int64
AST                int64
STL                int64
BLK                int64
TOV                int64
PF                 int64
PTS                int64
GmSc               int64
+/-                int64
WinOrLoss          int64
Margin             int64
dtype: object

In [11]:
jokic_full_df.drop(columns= ["MOV","GS"], axis=1, inplace = True)

In [12]:
display(jokic_full_df)

,Seconds_played,G,Home_or_Away,FG,FGA,FG%,3P,3PA,3P%,FT,...,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-,WinOrLoss,Margin
0,2005,1,1,12,17,1,1,3,0,2,...,6,3,0,3,4,27,24,-5,0,21
1,2064,2,1,7,13,1,1,2,0,11,...,10,0,0,5,4,26,25,-3,1,5
2,2321,3,0,6,10,1,1,1,1,6,...,13,1,1,3,2,19,25,18,1,5
3,1618,4,1,3,4,1,0,0,0,3,...,9,0,0,1,5,9,14,-10,0,25
4,2088,5,0,12,17,1,0,4,0,7,...,9,4,0,3,3,31,34,28,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2016,65,1,12,16,1,2,3,1,5,...,7,3,0,2,1,31,36,28,1,14
73,1913,66,0,10,20,0,1,6,0,10,...,11,1,0,3,2,31,28,19,1,23
74,2098,67,0,8,11,1,0,0,0,9,...,12,0,2,3,3,25,32,11,1,5
78,1481,68,1,7,13,1,0,2,0,0,...,4,2,3,8,1,14,10,-21,0,21


In [13]:
y = jokic_full_df['PTS'].values
display(y)

array([27, 26, 19,  9, 31, 12, 23, 15, 21, 26, 24, 29,  8, 31, 39, 19, 32,
       17, 24, 32, 19, 33, 31, 43, 25, 40, 13, 29, 41, 20, 40, 19, 30, 24,
       12, 28, 14, 21, 17, 36, 31, 25, 24, 26, 22, 14, 20, 29, 30, 27, 14,
       24, 15, 40, 14, 18, 17, 18, 37, 35, 28, 30, 24, 22, 31, 31, 25, 14,
        6])

In [40]:
X = jokic_full_df.drop(columns= 'PTS').values
display(X)

array([[2005,    1,    1, ...,   -5,    0,   21],
       [2064,    2,    1, ...,   -3,    1,    5],
       [2321,    3,    0, ...,   18,    1,    5],
       ...,
       [2098,   67,    0, ...,   11,    1,    5],
       [1481,   68,    1, ...,  -21,    0,   21],
       [1598,   69,    1, ...,   -5,    0,    4]])

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [42]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [43]:
number_input_features = 24

In [44]:
number_output_neurons = 1

In [45]:
hidden_nodes_layer1 = 8

In [46]:
hidden_nodes_layer2 = 4

In [47]:
nn = Sequential()

In [48]:
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

In [49]:
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [50]:
nn.add(Dense(units=1, activation="sigmoid"))

In [51]:
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 8)                 200       
                                                                 
 dense_6 (Dense)             (None, 4)                 36        
                                                                 
 dense_7 (Dense)             (None, 1)                 5         
                                                                 
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [52]:
nn.compile(loss= "binary_crossentropy", optimizer="adam", metrics=["mse"])

In [53]:
jokic_deep_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
2/2 [==============================] - 1s 4ms/step - loss: 12.9686 - mse: 683.1920
Epoch 2/50
2/2 [==============================] - 0s 14ms/step - loss: 12.2878 - mse: 682.8921
Epoch 3/50
2/2 [==============================] - 0s 10ms/step - loss: 11.6565 - mse: 682.6044
Epoch 4/50
2/2 [==============================] - 0s 13ms/step - loss: 11.1007 - mse: 682.3527
Epoch 5/50
2/2 [==============================] - 0s 17ms/step - loss: 10.4464 - mse: 682.0522
Epoch 6/50
2/2 [==============================] - 0s 5ms/step - loss: 9.8361 - mse: 681.7701
Epoch 7/50
2/2 [==============================] - 0s 13ms/step - loss: 9.2406 - mse: 681.4925
Epoch 8/50
2/2 [==============================] - 0s 13ms/step - loss: 8.7082 - mse: 681.2425
Epoch 9/50
2/2 [==============================] - 0s 10ms/step - loss: 8.1417 - mse: 680.9745
Epoch 10/50
2/2 [==============================] - 0s 6ms/step - loss: 7.5998 - mse: 680.7148
Epoch 11/50
2/2 [==============================] - 0s 8ms

In [54]:
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

1/1 - 0s - loss: -1.1102e-01 - mse: 568.4023 - 145ms/epoch - 145ms/step


In [55]:
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: -0.11102337390184402, Accuracy: 568.4022827148438


In [58]:
predictions = (nn.predict(X_test_scaled) > 0.5).astype("int32")

1/1 [==============================] - 0s 93ms/step


In [61]:
results = pd.DataFrame(predictions)

In [62]:
results

,0
0,1
1,0
2,1
3,0
4,0
5,0
6,1
7,0
8,1
9,0
